# SR-Net Image and SRM Normalization 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Libraries

In [ ]:
from scipy import misc, ndimage, signal
from sklearn.model_selection  import train_test_split
import numpy
import numpy as np
import random
import ntpath
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from keras import optimizers 
from keras import regularizers
import tensorflow as tf
import cv2
from keras import backend as K
from time import time
import time as tm
import datetime
from operator import itemgetter
import glob
from skimage.util.shape import view_as_blocks
from keras.utils import np_utils
from keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LSTM, SpatialDropout2D, Concatenate, Lambda,Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, UpSampling2D, BatchNormalization, ReLU

## 30 SRM filters for preprocessing and the activation function

In [ ]:
################################################## 30 SRM FILTERS
srm_weights = np.load('SRM_Kernels.npy')*(1/12) 
biasSRM=numpy.ones(30)
print (srm_weights.shape)
################################################## TLU ACTIVATION FUNCTION
T3 = 3;
def Tanh3(x):
    tanh3 = K.tanh(x)*T3
    return tanh3
##################################################

## SR-Net architecture

In [ ]:
def SR_Net(img_size=256):
    tf.keras.backend.clear_session()
    inputs = tf.keras.Input(shape=(img_size,img_size,1), name="input")
    # Layer 1
    conv0 =   tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), trainable=False, activation=Tanh3, use_bias=True)(inputs)
    conv1 =   tf.keras.layers.Conv2D(64, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(conv0)
    bn1 =     tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv1)
    # Layer 2
    conv2 =   tf.keras.layers.Conv2D(16, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(bn1)
    bn2 =     tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv2)
    # Layer 3
    conv3 =   tf.keras.layers.Conv2D(16, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(bn2)
    bn3 =     tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv3)
    conv4 =   tf.keras.layers.Conv2D(16, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(bn3)
    bn4 =     tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv4)
    skip1 =   tf.keras.layers.Add()([bn2, bn4])
    # Layer 4
    conv5 =   tf.keras.layers.Conv2D(16, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(skip1)
    bn5 =     tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv5)
    conv6 =   tf.keras.layers.Conv2D(16, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(bn5)
    bn6 =     tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv6)
    skip2 =   tf.keras.layers.Add()([skip1, bn6])
    # Layer 5
    conv7 =   tf.keras.layers.Conv2D(16, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(skip2)
    bn7 =     tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv7)
    conv8 =   tf.keras.layers.Conv2D(16, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(bn7)
    bn8 =     tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv8)
    skip3 =   tf.keras.layers.Add()([skip2, bn8])
    # Layer 6
    conv9 =   tf.keras.layers.Conv2D(16, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(skip3)
    bn9 =     tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv9)
    conv10 =  tf.keras.layers.Conv2D(16, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(bn9)
    bn10 =    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv10)
    skip4 =   tf.keras.layers.Add()([skip3, bn10])
    # Layer 7
    conv11 =  tf.keras.layers.Conv2D(16, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(skip4)
    bn11 =    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv11)
    conv12 =  tf.keras.layers.Conv2D(16, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(bn11)
    bn12 =    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv12)
    skip5 =   tf.keras.layers.Add()([skip4, bn12])
    # Layer 8
    conv13 =  tf.keras.layers.Conv2D(16, (1,1), strides=(2,2), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(skip5)
    bn13 =    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv13)
    conv14 =  tf.keras.layers.Conv2D(16, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(skip5)
    bn14 =    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv14)
    conv15 =  tf.keras.layers.Conv2D(16, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(bn14)
    bn15 =    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv15)
    pool1 =   tf.keras.layers.AveragePooling2D([3,3], (2,2), 'SAME', data_format='channels_last')(bn15)
    skip6 =   tf.keras.layers.Add()([bn13, pool1])
    # Layer 9
    conv16 =  tf.keras.layers.Conv2D(64, (1,1), strides=(2,2), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(skip6)
    bn16 =    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv16)
    conv17 =  tf.keras.layers.Conv2D(63, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(skip6)
    bn17 =    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv17)
    conv18 =  tf.keras.layers.Conv2D(64, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(bn17)
    bn18 =    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv18)
    pool2 =   tf.keras.layers.AveragePooling2D([3,3], (2,2), 'SAME', data_format='channels_last')(bn18)
    skip7 =   tf.keras.layers.Add()([bn16, pool2])
    # Layer 10
    conv19 =  tf.keras.layers.Conv2D(128, (1,1), strides=(2,2), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(skip7)
    bn19 =    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv19)
    conv20 =  tf.keras.layers.Conv2D(128, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(skip7)
    bn20 =    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv20)
    conv21 =  tf.keras.layers.Conv2D(128, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(bn20)
    bn21 =    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv21)
    pool3 =   tf.keras.layers.AveragePooling2D([3,3], (2,2), 'SAME', data_format='channels_last')(bn21)
    skip8 =   tf.keras.layers.Add()([bn19, pool3])
    # Layer 11
    conv22 =  tf.keras.layers.Conv2D(256, (1,1), strides=(2,2), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(skip8)
    bn22 =    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv22)
    conv23 =  tf.keras.layers.Conv2D(256, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(skip8)
    bn23 =    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv23)
    conv24 =  tf.keras.layers.Conv2D(256, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(bn23)
    bn24 =    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv24)
    pool4 =   tf.keras.layers.AveragePooling2D([3,3], (2,2), 'SAME', data_format='channels_last')(bn24)
    skip9 =   tf.keras.layers.Add()([bn22, pool4])
    # Layer 12
    conv25 =  tf.keras.layers.Conv2D(512, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(skip9)
    bn25 =    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv25)
    conv26 =  tf.keras.layers.Conv2D(512, (3,3), strides=(1,1), activation="relu", padding='SAME', kernel_initializer='glorot_normal')(bn25)
    bn26 =    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(conv26)
    GAP =     tf.keras.layers.GlobalAveragePooling2D(data_format = 'channels_last')(bn26)
    #Layer 13, FC, Softmax
    layers1 = tf.keras.layers.Dense(128,activation="relu")(GAP)
    layers1 = tf.keras.layers.Dropout(0.2)(layers1)
    layers1 = tf.keras.layers.Dense(64 ,activation="relu")(layers1)
    layers1 = tf.keras.layers.Dropout(0.2)(layers1)
    layers1 = tf.keras.layers.Dense(32 ,activation="relu")(layers1)
    
    #Softmax
    predictions = tf.keras.layers.Dense(2, activation="softmax", name="output")(layers1)
    
    #Model generation
    model = tf.keras.Model(inputs = inputs, outputs=predictions)
    
    #Optimizer
    optimizer=tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9) 
    
    #Compilator
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    print ("SR-net model generated")
    return model

## Defining different functions to work with the architecture

In [ ]:
def train(model, X_train, y_train, X_valid, y_valid, X_test, y_test, batch_size, epochs, model_name=""):
    start_time = tm.time()
    log_dir=path_log_base+"/"+model_name+"_"+str(datetime.datetime.now().isoformat()[:19].replace("T", "_").replace(":","-"))
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)
    filepath = log_dir+"/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=False, mode='max')
    model.reset_states()
    
    #VALORES EN TRAIN TEST Y VALIDACIÓN INICIALES, GRÁFICOS
    global lossTEST
    global accuracyTEST
    global lossTRAIN
    global accuracyTRAIN
    global lossVALID
    global accuracyVALID
    lossTEST,accuracyTEST   = model.evaluate(X_test, y_test,verbose=None)
    lossTRAIN,accuracyTRAIN = model.evaluate(X_train, y_train,verbose=None)
    lossVALID,accuracyVALID = model.evaluate(X_valid, y_valid,verbose=None)

    global history
    global model_Name
    global log_Dir
    model_Name = model_name
    log_Dir = log_dir
    
    history=model.fit(X_train, y_train, epochs=epochs, 
                      callbacks=[tensorboard,checkpoint], 
                      batch_size=batch_size,validation_data=(X_valid, y_valid),verbose=1)
    
    metrics = model.evaluate(X_test, y_test, verbose=0)
     
    TIME = tm.time() - start_time
    print("Time "+model_name+" = %s [seconds]" % TIME)
    
    print("\n")
    print(log_dir)
    return {k:v for k,v in zip (model.metrics_names, metrics)}

In [ ]:
def Final_Results_Test(model,PATH_trained_models):
    global AccTest
    global LossTest
    AccTest = []
    LossTest= [] 
    B_accuracy = 0 #B --> Best
    for filename in sorted(os.listdir(PATH_trained_models)):
      if filename != ('train') and filename != ('validation'):
        print(filename)
        model.load_weights(PATH_trained_models+'/'+filename)
        loss,accuracy = model.evaluate(X_test, y_test,verbose=0)
        print(f'Loss={loss:.4f} y Accuracy={accuracy:0.4f}'+'\n') 
        BandAccTest  = accuracy
        BandLossTest = loss
        AccTest.append(BandAccTest)    #Valores de la precisión en Test, para graficar junto a valid y train
        LossTest.append(BandLossTest)  #Valores de la perdida en Test, para graficar junto a valid y train
        if accuracy > B_accuracy:
          B_accuracy = accuracy
          B_loss = loss
          B_name = filename
    print("\n\nBest")
    print(B_name)
    print(f'Loss={B_loss:.4f} y Accuracy={B_accuracy:0.4f}'+'\n')

In [ ]:
from operator import itemgetter
def graphics(history, AccTest, LossTest, log_Dir, model_Name, lossTEST, lossTRAIN, lossVALID, accuracyTEST, accuracyTRAIN, accuracyVALID):
    numbers=AccTest
    numbers_sort = sorted(enumerate(numbers), key=itemgetter(1),  reverse=True)
    for i in range(int(len(numbers)*(0.05))): #5% Del total de las épocas
        index, value = numbers_sort[i]
        print("Test Accuracy {}, Época:{}\n".format(value, index+1))
    
    print("")
    
    numbers=history.history['accuracy']
    numbers_sort = sorted(enumerate(numbers), key=itemgetter(1),  reverse=True)
    for i in range(int(len(numbers)*(0.05))): #5% Del total de las épocas
        index, value = numbers_sort[i]
        print("Train Accuracy {}, Época:{}\n".format(value, index+1))
    
    print("")
    
    numbers=history.history['val_accuracy']
    numbers_sort = sorted(enumerate(numbers), key=itemgetter(1),  reverse=True)
    for i in range(int(len(numbers)*(0.05))): #5% Del total de las épocas
        index, value = numbers_sort[i]
        print("Validation Accuracy {}, Época:{}\n".format(value, index+1))

    with plt.style.context('seaborn-white'):
        plt.figure(figsize=(10, 10))
        #Plot training & validation accuracy values
        plt.plot(np.concatenate([np.array([accuracyTRAIN]),np.array(history.history['accuracy'])],axis=0))
        plt.plot(np.concatenate([np.array([accuracyVALID]),np.array(history.history['val_accuracy'])],axis=0))
        plt.plot(np.concatenate([np.array([accuracyTEST]),np.array(AccTest)],axis=0)) #Test
        plt.title('Accuracy Vs Epoch')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation', 'Test'], loc='upper left')
        plt.grid('on')
        plt.savefig(path_img_base+'/Accuracy_SR_Net_'+model_Name+'.eps', format='eps')
        plt.savefig(path_img_base+'/Accuracy_SR_Net_'+model_Name+'.svg', format='svg')
        plt.savefig(path_img_base+'/Accuracy_SR_Net_'+model_Name+'.pdf', format='pdf')     
        plt.show()
        
        plt.figure(figsize=(10, 10))
        #Plot training & validation loss values
        plt.plot(np.concatenate([np.array([lossTRAIN]),np.array(history.history['loss'])],axis=0))
        plt.plot(np.concatenate([np.array([lossVALID]),np.array(history.history['val_loss'])],axis=0))
        plt.plot(np.concatenate([np.array([lossTEST]),np.array(LossTest)],axis=0)) #Test
        plt.title('Loss Vs Epoch')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation', 'Test'], loc='upper left')
        plt.grid('on')
        plt.savefig(path_img_base+'/Loss_SR_Net_'+model_Name+'.eps', format='eps')
        plt.savefig(path_img_base+'/Loss_SR_Net_'+model_Name+'.svg', format='svg')
        plt.savefig(path_img_base+'/Loss_SR_Net_'+model_Name+'.pdf', format='pdf') 

In [ ]:
def top_models(AccTest,AccTrain,AccValid):
    numbers=AccTest
    numbers_sort = sorted(enumerate(numbers), key=itemgetter(1),  reverse=True)
    for i in range(int(len(numbers)*(0.05))): #5% total epochs
        index, value = numbers_sort[i]
        print("Test Accuracy {}, epoch:{}\n".format(value, index+1))
    
    print("")
    
    numbers=AccTrain
    numbers_sort = sorted(enumerate(numbers), key=itemgetter(1),  reverse=True)
    for i in range(int(len(numbers)*(0.05))): #5% total epochs
        index, value = numbers_sort[i]
        print("Train Accuracy {}, epoch:{}\n".format(value, index+1))
    
    print("")
    
    numbers=AccValid
    numbers_sort = sorted(enumerate(numbers), key=itemgetter(1),  reverse=True)
    for i in range(int(len(numbers)*(0.05))): #5% total epochs
        index, value = numbers_sort[i]
        print("Validation Accuracy {}, epoch:{}\n".format(value, index+1))

## Working with BOSSbase 1.01 WOW y PAYLOAD = 0.4bpp

In the README, there is a link to download the databases we use for the work. There is BOSSbase 1.01, cover images and stego. The steganographic algorithms used in the paper are WOW and S-UNIWARD, with a payload of 0.4bpp.

Note: If you want to train the algorithm with S-UNIWARD 0.4 bpp, change "PATH04_WOW1" and  "base_name".

# Image Normalization

Choose any of the five image normalizations and train the model. 

## [0,255]

In [ ]:
PATH04 = 'drive/My Drive/Databases/Payload_04bpp'

PATH04_WOW1 = "/WOW_04bpp_BOSS/"

#Train
X_train = np.load(PATH04+PATH04_WOW1+'X_train.npy')
y_train = np.load(PATH04+PATH04_WOW1+'y_train.npy')
#Valid
X_valid = np.load(PATH04+PATH04_WOW1+'X_valid.npy')
y_valid = np.load(PATH04+PATH04_WOW1+'y_valid.npy')
#Test
X_test = np.load(PATH04+PATH04_WOW1+'X_test.npy')
y_test = np.load(PATH04+PATH04_WOW1+'y_test.npy')

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)


## [-12,8]

In [ ]:
PATH04 = 'drive/My Drive/Databases/Payload_04bpp'

PATH04_WOW1 = "/WOW_04bpp_BOSS/"

#Train
X_train = np.load(PATH04+PATH04_WOW1+'X_train.npy')
y_train = np.load(PATH04+PATH04_WOW1+'y_train.npy')
#Valid
X_valid = np.load(PATH04+PATH04_WOW1+'X_valid.npy')
y_valid = np.load(PATH04+PATH04_WOW1+'y_valid.npy')
#Test
X_test = np.load(PATH04+PATH04_WOW1+'X_test.npy')
y_test = np.load(PATH04+PATH04_WOW1+'y_test.npy')

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)

X_train = (X_train-X_train.min())/(X_train.max()-X_train.min())*20-12
X_valid = (X_valid-X_valid.min())/(X_valid.max()-X_valid.min())*20-12
X_test = (X_test-X_test.min())/(X_test.max()-X_test.min())*20-12
print(X_train.min())
print(X_train.max())

## [0,1]

In [ ]:
PATH04 = 'drive/My Drive/Databases/Payload_04bpp'

PATH04_WOW1 = "/WOW_04bpp_BOSS/"

#Train
X_train = np.load(PATH04+PATH04_WOW1+'X_train.npy')
y_train = np.load(PATH04+PATH04_WOW1+'y_train.npy')
#Valid
X_valid = np.load(PATH04+PATH04_WOW1+'X_valid.npy')
y_valid = np.load(PATH04+PATH04_WOW1+'y_valid.npy')
#Test
X_test = np.load(PATH04+PATH04_WOW1+'X_test.npy')
y_test = np.load(PATH04+PATH04_WOW1+'y_test.npy')

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)

X_train = (X_train-X_train.min())/(X_train.max()-X_train.min())*1-0
X_valid = (X_valid-X_valid.min())/(X_valid.max()-X_valid.min())*1-0
X_test = (X_test-X_test.min())/(X_test.max()-X_test.min())*1-0
print(X_train.min())
print(X_train.max())
print(X_valid.min())
print(X_valid.max())
print(X_test.min())
print(X_test.max())

## [-1,1]

In [ ]:
PATH04 = 'drive/My Drive/Databases/Payload_04bpp'

PATH04_WOW1 = "/WOW_04bpp_BOSS/"

#Train
X_train = np.load(PATH04+PATH04_WOW1+'X_train.npy')
y_train = np.load(PATH04+PATH04_WOW1+'y_train.npy')
#Valid
X_valid = np.load(PATH04+PATH04_WOW1+'X_valid.npy')
y_valid = np.load(PATH04+PATH04_WOW1+'y_valid.npy')
#Test
X_test = np.load(PATH04+PATH04_WOW1+'X_test.npy')
y_test = np.load(PATH04+PATH04_WOW1+'y_test.npy')

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)

X_train = (X_train-X_train.min())/(X_train.max()-X_train.min())*2-1
X_valid = (X_valid-X_valid.min())/(X_valid.max()-X_valid.min())*2-1
X_test = (X_test-X_test.min())/(X_test.max()-X_test.min())*2-1
print(X_train.min())
print(X_train.max())

## [-0.5,0.5]

In [ ]:
PATH04 = 'drive/My Drive/Databases/Payload_04bpp'

PATH04_WOW1 = "/WOW_04bpp_BOSS/"

#Train
X_train = np.load(PATH04+PATH04_WOW1+'X_train.npy')
y_train = np.load(PATH04+PATH04_WOW1+'y_train.npy')
#Valid
X_valid = np.load(PATH04+PATH04_WOW1+'X_valid.npy')
y_valid = np.load(PATH04+PATH04_WOW1+'y_valid.npy')
#Test
X_test = np.load(PATH04+PATH04_WOW1+'X_test.npy')
y_test = np.load(PATH04+PATH04_WOW1+'y_test.npy')

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)

X_train = (X_train-X_train.min())/(X_train.max()-X_train.min())*1-0.5
X_valid = (X_valid-X_valid.min())/(X_valid.max()-X_valid.min())*1-0.5
X_test = (X_test-X_test.min())/(X_test.max()-X_test.min())*1-0.5
print(X_train.min())
print(X_train.max())

## CNN name and algorithm 

In [ ]:
path_log_base = './logs/SR-Net/WOW'
path_img_base = './image/SR-Net/WOW'

## Training

In [ ]:
model= SR_Net() 
train(model, X_train, y_train, X_valid, y_valid, X_test, y_test, batch_size=32, epochs=100, model_name="SR-Net.....")

## Test and Validation

In [ ]:
model= SR_Net() 
PATH_trained_models = ""
Final_Results_Test(model,PATH_trained_models)

## Training, validation and testing graph

In [ ]:
graphics(history, AccTest, LossTest, PATH_trained_models, model_Name, lossTEST, lossTRAIN, lossVALID, accuracyTEST, accuracyTRAIN, accuracyVALID)

## Top

In [ ]:
top_models(AccTest, accuracyTRAIN, accuracyVALID)

Note: If you want to train the algorithm with S-UNIWARD 0.4 bpp, change "PATH04_WOW1" and  "base_name".